In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
stations = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Bonus Challenge Assignment: Temperature Analysis II

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set
year_temps = calc_temps('2017-01-01', '2017-12-31')

In [ ]:
year_temps

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
ptp = year_temps[0][2] - year_temps[0][0]
ptp

In [ ]:
plt.bar(1,year_temps[0][1], yerr= ptp, color='salmon')
plt.title("Trip Avg Temp")
plt.ylabel("Temp (F)")
plt.gca().axes.get_xaxis().set_visible(False)
plt.show()

### Daily Rainfall Average

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
sel = [Measurement.prcp, stations.station, stations.name, stations.latitude, stations.longitude, stations.elevation]
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
daily_precip = session.query(*sel).filter(Measurement.station == stations.station).filter(Measurement.date <= '2017-04-20').\
                        filter(Measurement.date >= '2017-04-14').order_by(Measurement.prcp.desc())

In [ ]:
daily_rain_df = pd.read_sql(daily_precip.statement, engine)

In [ ]:
daily_rain_df.head()

In [ ]:
total_rain_gdf = daily_rain_df.groupby(['station','name','latitude','longitude','elevation'])['prcp'].sum()
total_rain_gdf

### Daily Temperature Normals

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = dt.datetime.strptime('2017-04-14', '%Y-%m-%d')
end_date = dt.datetime.strptime('2017-04-20', '%Y-%m-%d')

In [ ]:
start_date

In [ ]:
trip_days = end_date-start_date

# Use the start and end date to create a range of dates
dates = pd.date_range(start_date, end_date)
Dates = dates.strftime("%Y-%m-%d").tolist() #for dataframe

In [ ]:
# Strip off the year and save a list of strings in the format %m-%d
trip_dates = dates.strftime("%m-%d").tolist()

In [ ]:
# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.
normals = []

for day in trip_dates:
    norm = daily_normals(day)[0]
    normals.append(norm)    

In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.DataFrame(data=normals, columns=['TMIN', 'TAVG', 'TMAX'] ,index = Dates)

In [ ]:
normals_df

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
normals_df.plot.area(figsize=(6,4), stacked=False, alpha=0.3)
plt.xlabel("Dates")
plt.ylabel("Temperature")
plt.xticks(np.arange(len(trip_dates)),Dates, rotation=30)
plt.show()

## Close Session

In [ ]:
session.close()